In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf


from bs4 import BeautifulSoup
import requests

2023-05-21 21:51:25.815309: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-21 21:51:25.816875: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-21 21:51:25.853612: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-21 21:51:25.855687: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-21 21:51:26.468197: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
# Function to scrape clothing item details from a website
def scrape_clothing_details(url, class_name):
    response = requests.get(url)
    html_content = response.content
    soup = BeautifulSoup(html_content, "html.parser")
    items = soup.find_all("div", class_=class_name)
    
    clothing_details = []
    for item in items:
        link = item.find("a")["href"] if item.find("a") else ""
        image = item.find("img")
        image_url = image["src"] if image else ""
        image_title = image["title"] if image and "title" in image.attrs else ""
        text = item.find("p").get_text(strip=True) if item.find("p") else ""
        text_with_title = f"{image_title} {text}"
        clothing_details.append({"link": link, "image_url": image_url, "text": text_with_title})
    
    return clothing_details

In [3]:
url = "https://www.bewakoof.com/men-clothing"

item_desc = scrape_clothing_details(url, "plp-product-card")
print(len(item_desc))

10


In [4]:
# Specify the base URL of the website to scrape
base_url_m = "https://www.bewakoof.com/men-clothing"

base_url_w = "https://www.bewakoof.com/women-clothing"

# Specify the class name of the elements containing clothing details
class_name = "plp-product-card"

# Scrape clothing details from multiple pages
all_details_men = []
all_details_women = []

for page_num in range(1, 51):  # Scraping 50 pages
    # men
    url_men = f"{base_url_m}?page={page_num}"
    page_details = scrape_clothing_details(url_men, class_name)
    all_details_men.extend(page_details)

    # women
    url_women = f"{base_url_w}?page={page_num}"
    page_details = scrape_clothing_details(url_women, class_name)
    all_details_women.extend(page_details)

In [5]:
all_details_men[0]

{'link': '/p/men-friends-logo-printed-sweatshirt-men',
 'image_url': 'https://images.bewakoof.com/t640/men-friends-logo-printed-sweatshirt-557029-1670918374-1.jpg',
 'text': "Men's Red Friends Logo Typography Sweatshirt-Front Bewakoof "}

In [6]:
all_details_women[0]

{'link': '/p/women-friends-logo-printed-sweatshirt',
 'image_url': 'https://images.bewakoof.com/t640/women-s-black-friends-logo-typography-oversized-sweatshirt-557028-1672828315-1.jpg',
 'text': "Women's Black Friends Logo Typography Oversized Sweatshirt-Front Bewakoof "}

In [7]:
final_df = all_details_men + all_details_women

In [8]:
for map in final_df:
    map["link"] = "https://www.bewakoof.com" + map["link"]

In [9]:
final_df[12]

{'link': 'https://www.bewakoof.com/p/mens-black-hooded-jacket-3',
 'image_url': 'https://images.bewakoof.com/t640/men-s-black-hooded-jacket-545758-1664352540-1.jpg',
 'text': "Men's Black Hooded Jacket-Front Bewakoof "}

In [10]:
final_df = pd.DataFrame(final_df)

In [11]:
final_df.to_csv("fashion_data_test.csv", index=None)